# Load point values

This is a demo that queries a list of points and orders by risk according to Wildfire Risk based on the [Wildfire Forecast](https://data.spatiafi.com/dataset/36) dataset.  We start by loading our App Credentials, and then using the [point](https://docs.spatiafi.com/api/#operation/point_api_point__lon___lat__get) endpoint to fetch fire risk values at the points of interest.

---

Install `spatiafi` (if not already installed):

In [ ]:
!pip install spatiafi

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import spatiafi
import xarray as xr

In [ ]:
session = spatiafi.get_session()

In [ ]:
session

# NOTEBOOK THAT GETS 
- POINT VALUE (list of points at risk of __)
- POINT VALUE OVER TIME (how that's changing over time for top performer)
- LINES (TRANSMISSION LINE/ROAD? take line and dilate convert to thicker polygon)
- POLYGONS (SPATIAL AVERAGE)

## Point Value
This section takes a list of example asset locations (lats/lons) and queries the day 1 forecast of FWI at each location. This should give us an understanding of the near-term fire risk for our assets

In [ ]:
lon = ["-104.18", "-92.88", "-113.46", "-122.33", "-74.00", "-89.44", "25.66", "137.41"]
lat = ["55.48", "55.74", "56.99", "47.61", "40.71", "31.01", "23.41", "-24.57"]

values = np.zeros(len(lat))

for i in range(len(lon)):
    url = "https://api.spatiafi.com/api/point/" + lon[i] + "," + lat[i]

    query = {"item_id": "ce-cfsv2-era5-fwi-forecast-day01", "bidx": "1"}

    response = session.get(url, params=query)

    data = response.json()

    values[i] = data["values"][0]

    print(data)

We now plot these values so that we can more clearly see which assets are of greatest risk.

In [ ]:
lon_double = [float(x) for x in lon]
lat_double = [float(x) for x in lat]

data = xr.DataArray(values, dims=("points",), coords={"points": range(len(values))})

# Add latitude and longitude as coordinates
data = data.assign_coords(
    latitude=("points", lat_double), longitude=("points", lon_double)
)

fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the data using scatter plot
plt.scatter(
    data["longitude"],
    data["latitude"],
    c=data,
    cmap="viridis",
    transform=ccrs.PlateCarree(),
)

# Add coastlines
ax.coastlines()

# Add colorbar and labels
# plt.colorbar(label='Values')
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# Show the plot
plt.show()

We can see that the assets with the largest FWI June 12th forecast are in __.

## Point Values Over Time
We now include an example that queries timeseries from the same point values. This function can be used to, for example, track trends in the FWI to see if there will be assets we should be concerned about in a few days' time.

In [ ]:
for i in range(len(lon)):
    url = "https://api.spatiafi.com/api/timeseries/point/" + lon[i] + "," + lat[i]

    query = {
        "coll_id": "climate-engine-cfsv2-era5-land-fire-weather-index-forecasts",
        "datetime": "2023-06-11/2023-06-15",
        "bidx": "1",
    }

    response = session.get(url, params=query)

    data = response.json()

    all_values = []
    for key in data:
        all_values.extend(data[key]["values"])

    # Convert the list of values to a NumPy array
    values_array = np.array(all_values)

    # Print the array
    print(values_array)

In [ ]:
# SET VALUES LESS THAN 1 TO 0, LESS THAN 10 TO 1, LESS THAN 30 TO 2, LESS THAN 75 TO 3, GREATER THAN 75 TO 4